# Part 8: Federated Learning with Encrypted Gradient Aggregation

In the last few sections, we've been learning about encrypted computation by building several simple programs (including a database and a ledger). In this section, we're going to return to machine learning by using our new tools for encrypted computation for Federated Learning.

In the last Federated Learning Demo (Part 4), we had a "trusted aggregator" who was responsible for averaging the model updates from multiple workers. This is less than ideal because it assumes that we can find someone trustworthy enough to have access to this sensitive information. This is not always the case.

Thus, in this notebook, we will show how one can use SMPC to perform aggregation such that we don't need a "trusted aggregator".

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Theo Ryffel - Github: [LaRiffle](https://github.com/LaRiffle)

# Section 1: Normal Federated Learning

First I'll show some code which performs normal federated learning on the Boston Housing Dataset. This section of code is broken into several sections.

### Setting Up

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

print(torch.__version__)
# Training settings
parser = argparse.ArgumentParser(description='PyTorch Example')
parser.add_argument('--batch-size', type=int, default=8, metavar='N',
                    help='input batch size for training (default: 8)')
parser.add_argument('--test-batch-size', type=int, default=8, metavar='N',
                    help='input batch size for testing (default: 8)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                    help='learning rate (default: 0.001)')
parser.add_argument('--momentum', type=float, default=0.0, metavar='M',
                    help='SGD momentum (default: 0.0)')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args([])

torch.manual_seed(args.seed)
kwargs = {}

0.3.1


# Loading the Dataset

In [2]:
import pickle
f = open('../other/data/boston_housing.pickle','rb')
((X, y), (X_test, y_test)) = pickle.load(f)
f.close()

X = torch.from_numpy(X).type(torch.FloatTensor)
y = torch.from_numpy(y).type(torch.FloatTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.FloatTensor)
# preprocessing
mean = X.mean(0, keepdim=True)
dev = X.std(0, keepdim=True)
mean[:, 3] = 0. # the feature at column 3 is binary,
dev[:, 3] = 1.  # so I'd rather not standardize it
X = (X - mean) / dev
X_test = (X_test - mean) / dev
train = TensorDataset(X, y)
test = TensorDataset(X_test, y_test)
train_loader = DataLoader(train, batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(test, batch_size=args.test_batch_size, shuffle=True, **kwargs)

# Neural Network Structure

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc3 = nn.Linear(24, 1)

    def forward(self, x):
        x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()


optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

# Hookinfg PyTorch

In [4]:
import syft
import syft as sy
from syft.core import utils
import torch
import torch.nn.functional as F
import json
import random
from syft.core.frameworks.torch import utils as torch_utils
from torch.autograd import Variable
hook = sy.TorchHook(verbose=False)
me = hook.local_worker
bob = sy.VirtualWorker(id="bob",hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice",hook=hook, is_client_worker=False)
me.is_client_worker = False

compute_nodes = [bob, alice]

bob.add_workers([alice])
alice.add_workers([bob])


**Send data to the worker** <br>
Usually they would already have it, this is just for demo

In [6]:
train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = Variable(data)
    target = Variable(target.float())
    data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target.send(compute_nodes[batch_idx % len(compute_nodes)])
    train_distributed_dataset.append((data, target))

## Training Function

In [7]:
def train(epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_distributed_dataset):
            
        worker = data.location
        model.send(worker)

        optimizer.zero_grad()
        # update the model
        pred = model(data)
        loss = F.mse_loss(pred, target.float())
        loss.backward()
        model.get()
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.data[0]))
        


# Testing Function

In [8]:
def test():
    model.eval()
    test_loss = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.mse_loss(output, target.float(), size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))


# Training the Model

In [9]:
import time

In [10]:
%%time
t = time.time()
args.epochs = 10
torch.encode_timer = 0
torch.handle_call_timer = 0
torch.execute_call_timer = 0

for epoch in range(1, args.epochs + 1):
    train(epoch)

    
total_time = time.time() - t
print('Encoding', round(torch.encode_timer, 2), 's', round(torch.encode_timer/total_time*100, 2), '%')
print('Handling', round(torch.handle_call_timer, 2), 's',  round(torch.handle_call_timer/total_time*100, 2), '%')
print('Execute call', round(torch.execute_call_timer, 2), 's',  round(torch.execute_call_timer/total_time*100, 2), '%')
print('Total', round(total_time, 2), 's')

Train Epoch: 1 [0/408 (0%)]	Loss: 562.255920
Train Epoch: 1 [80/408 (20%)]	Loss: 551.988525
Train Epoch: 1 [160/408 (39%)]	Loss: 640.131042
Train Epoch: 1 [240/408 (59%)]	Loss: 113.095512
Train Epoch: 1 [320/408 (78%)]	Loss: 17.793676
Train Epoch: 1 [400/408 (98%)]	Loss: 36.492382
Train Epoch: 2 [0/408 (0%)]	Loss: 8.652936
Train Epoch: 2 [80/408 (20%)]	Loss: 17.827301
Train Epoch: 2 [160/408 (39%)]	Loss: 15.696460
Train Epoch: 2 [240/408 (59%)]	Loss: 16.655910
Train Epoch: 2 [320/408 (78%)]	Loss: 17.403427
Train Epoch: 2 [400/408 (98%)]	Loss: 17.845072
Train Epoch: 3 [0/408 (0%)]	Loss: 7.068494
Train Epoch: 3 [80/408 (20%)]	Loss: 12.011536
Train Epoch: 3 [160/408 (39%)]	Loss: 7.905757
Train Epoch: 3 [240/408 (59%)]	Loss: 13.399664
Train Epoch: 3 [320/408 (78%)]	Loss: 12.010727
Train Epoch: 3 [400/408 (98%)]	Loss: 10.196638
Train Epoch: 4 [0/408 (0%)]	Loss: 6.434998
Train Epoch: 4 [80/408 (20%)]	Loss: 7.154055
Train Epoch: 4 [160/408 (39%)]	Loss: 5.233717
Train Epoch: 4 [240/408 (59%)]	

# Calculating Performance

In [11]:
test()


Test set: Average loss: 20.7802



# Section 2: Adding Encrypted Aggregation

Now we're going to modify this example slightly to aggregate gradients using encryption. The main piece that's different is really 1 or 2 lines of code in the train() function, which I'll point out. For the moment, let's re-preocess our data and initialize a model for bob and alice.

In [26]:
remote_dataset = (list(),list())

for batch_idx, (data,target) in enumerate(train_loader):
    data = Variable(data)
    target = Variable(target.float())
    data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target.send(compute_nodes[batch_idx % len(compute_nodes)])
    remote_dataset[batch_idx % len(compute_nodes)].append((data, target))

def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    pred = model(data)
    loss = F.mse_loss(pred, target.float())
    loss.backward()
    optimizer.step()
    return model

bobs_model = Net()
alices_model = Net()

bobs_optimizer = optim.SGD(bobs_model.parameters(), lr=args.lr, momentum=args.momentum)
alices_optimizer = optim.SGD(alices_model.parameters(), lr=args.lr, momentum=args.momentum)

models = [bobs_model, alices_model]
params = [list(bobs_model.parameters()), list(alices_model.parameters())]
optimizers = [bobs_optimizer, alices_optimizer]


## Building our Training Logic

The only **real** difference is inside of this train method. Let's walk through it step-by-step.

### Part A: Train:

In [30]:
# this is selecting which batch to train on
data_index = 0


# update remote models
# we could iterate this multiple times before proceeding, but we're only iterating once per worker here
for remote_index in range(len(compute_nodes)):
    data, target = remote_dataset[remote_index][data_index]
    models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])


### Part B: Encrypted Aggregation

In [31]:
# create a list where we'll deposit our encrypted model average
new_params = list()

In [32]:
# iterate through each parameter
for param_i in range(len(params[0])):

    # for each worker
    spdz_params = list()
    for remote_index in range(len(compute_nodes)):
        
        # select the identical parameter from each worker and copy it
        copy_of_parameter = params[remote_index][param_i].data+0
        
        # since SMPC can only work with integers (not floats), we need
        # to use Integers to store decimal information. In other words,
        # we need to use "Fixed Precision" encoding.
        # fix it's precision (read more about Fixed Precision encodings)
        fixed_precision_param = copy_of_parameter.fix_precision()
        
        # now we encrypt it on the remote machine. Note that 
        # fixed_precision_param is ALREADY a pointer. Thus, when
        # we call share, it actually encrpyts the data that the
        # data is pointing TO. This returns a POINTER to  the 
        # MPC Shared object, which we need to fetch.
        encrypted_param = fixed_precision_param.share(bob, alice)
        
        # now we fetch the pointer to the MPC shared value
        param = encrypted_param.get()
        
        # save the parameter so we can average it with the same parameter
        # from the other workers
        spdz_params.append(param)

    # average params from multiple workers, fetch them to the local machine
    # decrypt and decode (from fixed precision) back into a floaing point number
    new_param = (spdz_params[0] + spdz_params[1]).get().decode()/2
    
    # save the new averaged parameter
    new_params.append(new_param)


### Part C: Cleanup

In [33]:
for model in params:
    for param in model:
        param.data *= 0

for model in models:
    model.get()

for remote_index in range(len(compute_nodes)):
    for param_index in range(len(params[remote_index])):
        params[remote_index][param_index].data.set_(new_params[param_index])

## Let's put it all Together!!

And now that we know each step, we can put it all together into one training loop!

In [34]:

def train(epoch):

    for data_index in range(len(remote_dataset[0])-1):
        # update remote models
        for remote_index in range(len(compute_nodes)):
            data, target = remote_dataset[remote_index][data_index]
            models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])

        new_params = list()

        for param_i in range(len(params[0])):

            spdz_params = list()
            for remote_index in range(len(compute_nodes)):
                spdz_params.append((params[remote_index][param_i].data+0).fix_precision().share(bob, alice).get())

            new_param = (spdz_params[0] + spdz_params[1]).get().decode()/2
            new_params.append(new_param)

        for model in params:
            for param in model:
                param.data *= 0

        for model in models:
            model.get()

        for remote_index in range(len(compute_nodes)):
            for param_index in range(len(params[remote_index])):
                params[remote_index][param_index].data.set_(new_params[param_index])

In [35]:
def test():
    models[0].eval()
    test_loss = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = models[0](data)
        test_loss += F.mse_loss(output, target.float(), size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))


In [29]:
%%time
t = time.time()
args.epochs = 10
torch.encode_timer = 0
torch.handle_call_timer = 0
torch.execute_call_timer = 0

for epoch in range(1, args.epochs + 1):
    print(epoch)
    train(epoch)
    test()

    
total_time = time.time() - t
print('Encoding', round(torch.encode_timer, 2), 's', round(torch.encode_timer/total_time*100, 2), '%')
print('Handling', round(torch.handle_call_timer, 2), 's',  round(torch.handle_call_timer/total_time*100, 2), '%')
print('Execute call', round(torch.execute_call_timer, 2), 's',  round(torch.execute_call_timer/total_time*100, 2), '%')
print('Total', round(total_time, 2), 's')

1

Test set: Average loss: 555.1237

2

Test set: Average loss: 153.6630

3

Test set: Average loss: 26.8698

4

Test set: Average loss: 21.4207

5

Test set: Average loss: 19.2581

6

Test set: Average loss: 17.9186

7

Test set: Average loss: 17.1278

8

Test set: Average loss: 16.8299

9

Test set: Average loss: 16.6942

10

Test set: Average loss: 16.7554

Encoding 0 s 0.0 %
Handling 0 s 0.0 %
Execute call 0 s 0.0 %
Total 121.25 s
CPU times: user 1min 14s, sys: 9.97 s, total: 1min 24s
Wall time: 2min 1s


# Congratulations!!! - Time to Join the Community!

Congraulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on Github

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft Github Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for github issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)